In [0]:
"""
  File setup
"""
# from google.colab import drive
# drive.mount("/content/drive")

# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/Proj4/')

In [0]:
"""
  Import packages
"""
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [0]:
"""
Load Fashion MINST set
"""
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

tmp = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transforms.ToTensor())
n = len(tmp)
idx = list(range(n))  
np.random.shuffle(idx)
val_idx = idx[:n//2]
test_idx = idx[n//2:]
testset = torch.utils.data.Subset(tmp, test_idx)
validset = torch.utils.data.Subset(tmp, val_idx)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=True, num_workers=2)

classes = range(10)
classes_names = ("T-shirt/top", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle boot")


In [0]:
"""
Display sample images
"""
def imshow(img):
    npimg = img.numpy()
    print(npimg.shape)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes_names[labels[j]] for j in range(4))) # print labels

In [0]:
"""
Define neural network and hyperparameters
"""
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 400)
        self.fc2 = nn.Linear(400, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

In [0]:
"""
Save and load functions
"""
def save(net, epoch, path):
  torch.save({
            'epoch': epoch,
            'net_state_dict': net.state_dict()
            }, path)
  
def load(path):
  net = Net()
  state = torch.load(path)
  net.load_state_dict(state['net_state_dict'])
  epoch = state['epoch']
  return net, epoch


In [6]:
"""
Train model
"""
n=100
validation_vec = np.zeros(n)
training_vec = np.zeros(n)
for epoch in range(30): 
    running_loss = 0.0
    print("EPOCH {}".format(epoch+1))
    for data in tqdm(trainloader, position=0, leave=True):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    validation_vec[epoch], _ = validate(net, validloader, 10)   
    training_vec[epoch], _ = validate(net, trainloader, 10)
    save(net, epoch, "./p1-channels64-relu/p1-" + str(epoch+1))

print('Finished Training')

  0%|          | 0/15000 [00:00<?, ?it/s]

EPOCH 1


 20%|█▉        | 2993/15000 [00:11<00:42, 279.63it/s]

KeyboardInterrupt: ignored

In [7]:
"""
Test the net by class
"""
def validate(net, testset, n, by_class=False):
  correct_by_class = np.zeros(n)
  total_by_class = np.zeros(n)
  correct = 0
  total = 0
  with torch.no_grad():
    for data in tqdm(testset, position=0, leave=True):
      images, labels = data[0].to(device), data[1].to(device)
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      if by_class:
        for j in range(4):
            correct_by_class[labels[j]] += (labels[j] == predicted[j])
            total_by_class[labels[j]] += 1 
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  return correct / total, correct_by_class / total_by_class

net, _ = load("./p1-channels64-relu/p1-30")
net.to(device)
print(validate(net, testloader, 10, True))

 33%|███▎      | 409/1250 [00:01<00:03, 255.85it/s]

KeyboardInterrupt: ignored

In [0]:
n=30
epochs = np.linspace(1, n, num=n, endpoint=True)
validation_vec = np.zeros(n)
test_vec = np.zeros(n)
training_vec = np.zeros(n)
for i in range(n):
  print(i)
  net, _ = load("./p1-channels64-relu/p1-" + str(i+1))
  net.to(device)
  score, _  = validate(net, testloader, 10)
  validation_vec[i] = score
  score, _  = validate(net, trainloader, 10)
  training_vec[i] = score

In [0]:
"""
PLOT VALIDATION VS TRAINING
"""
fig = plt.figure(figsize=(10,5))
plt.plot(epochs, validation_vec, label="validation")
plt.plot(epochs, training_vec, label="train")
plt.title("Validation vs Training Accuracy")
plt.xlabel("epoch")
plt.ylabel("score")
fig.legend()
plt.show()

In [0]:
"""
Visualize Filters
"""
net = load("./p1-channels64-relu/p1-30")[0]
weights1 = net.conv1.weight.data.numpy()
weights2 = net.conv2.weight.data.numpy()

fig = plt.figure(figsize=(10, 10))
axes = fig.subplots(nrows=8, ncols=8)
for i in range(64):
  axes[i // 8][i % 8].imshow(weights1[i, 0])
print(weights1.shape)
print(weights2.shape)
plt.show()

fig = plt.figure(figsize=(10, 10))
axes = fig.subplots(nrows=8, ncols=8)
for i in range(64):
  axes[i // 8][i % 8].imshow(weights2[i, 0])
plt.show()

In [0]:
net.to(device)
print(validate(net, testloader, 10, True))
print(validate(net, validloader, 10, True))

In [0]:
figure, axes = plt.subplots(nrows=10, ncols=4, figsize=(20, 20))
found = np.zeros(40)
net = load("./p1-channels64-relu/p1-30")[0]
for data in testloader:
  images, labels = data
  outputs = net(images)
  _, predicted = torch.max(outputs.data, 1)
  for j in range(4):
    if labels[j] == predicted[j]:
      if not found[labels[j]*4]:
        found[labels[j]*4] = 1
        axes[labels[j]][0].imshow(images[j,0].numpy())
        axes[labels[j]][0].set_title("CORRECTLY LABELLED: {}".format(classes_names[labels[j]]))
      elif not found[labels[j]*4+1]:
        found[labels[j]*4+1] = 1
        axes[labels[j]][1].imshow(images[j,0].numpy())
        axes[labels[j]][1].set_title("CORRECTLY LABELLED: {}".format(classes_names[labels[j]]))
    if labels[j] != predicted[j]:
      if not found[labels[j]*4+2]:
        found[labels[j]*4+2] = 1
        axes[labels[j]][2].imshow(images[j,0].numpy())
        axes[labels[j]][2].set_title("INCORRECTLY LABELLED: {}".format(classes_names[predicted[j]]))
      elif not found[labels[j]*4+3]:
        found[labels[j]*4+3] = 1
        axes[labels[j]][3].imshow(images[j,0].numpy())
        axes[labels[j]][3].set_title("INCORRECTLY LABELLED: {}".format(classes_names[predicted[j]]))
figure.tight_layout(pad=2.0)